In [11]:
import torch
from torch.nn import functional as F

In [2]:
import NanoGPTLangugageModel as nano
device = nano.device

In [3]:
model = nano.NanoGPTLanguageModel()
model.load_state_dict(torch.load('model_weights.pth'))
m = model.to(nano.device)
encode, decode = nano.encode, nano.decode

In [4]:
start_str = "The"
idx = torch.tensor(encode(start_str), dtype=torch.long, device=device).unsqueeze(0)
print(decode(m.generate(idx = idx, max_new_tokens=500)[0].tolist()))

Theeeeeet-nis, and, eabish; se thag we
I
I!
GLULICIrI
Lork:
An t sour for tblly kee qors tone ror'd,
I I mo yor wath
BEome the tot sius mis figese chat,
A tha there fore ie
I, say our matn wotte foir nd, timy
B he me the my y fean;
I sr'wid ad, yuct frile detiret!
Rwe mer.
WIFat IRGi'dethe wThe
CKI
Hest gow to le ther thene not
Merd come that!
My four, heu piss whe? pre maws Mou bone
The
Khe thatsin, the le, beatl

LANCLININNBNT: I:
Tho seener migh s,
Thit
To vos a tror you for.
Lorvove bot se
Sove


In [70]:
q = torch.arange(0, 24)
k = torch.arange(0, 24) 
v = torch.arange(0, 24) + 6
q, k, v = [torch.reshape(x, (2, 2, 6)) for x in (q, k, v)] # B, T, C
q, k, v = [torch.reshape(x*1.0, (2, 2, 2, 3)) for x in (q, k, v)] # B, T, n, h

In [71]:
wei = torch.einsum('bqnh,bknh->bqkn', q, k) # B, Q, K, n
wei = F.softmax(wei, dim=-2)
out = torch.einsum('bqkn,bknh->bqnh', wei, v) # B, n, Q, h
out

tensor([[[[12., 13., 14.],
          [15., 16., 17.]],

         [[12., 13., 14.],
          [15., 16., 17.]]],


        [[[24., 25., 26.],
          [27., 28., 29.]],

         [[24., 25., 26.],
          [27., 28., 29.]]]])

In [51]:
q, k, v = [torch.transpose(x, 1, 2) for x in (q, k, v)] # B, n, T, h
wei = torch.einsum('bnqh,bnkh->bnqk', q, k) # B, n, Q, K
wei = F.softmax(wei, dim=-1)
out = torch.einsum('bnqk,bnkh->bnqh', wei, v) # B, n, Q, h
out


tensor([[[[12., 13., 14.],
          [12., 13., 14.]],

         [[15., 16., 17.],
          [15., 16., 17.]]],


        [[[24., 25., 26.],
          [24., 25., 26.]],

         [[27., 28., 29.],
          [27., 28., 29.]]]])

In [72]:
# torch.trill experiment
seq_len = wei.size(1)
tril = torch.tril(torch.ones((2, 2), dtype=torch.bool))
mask = torch.tril(torch.ones(1, 1, seq_len, seq_len), diagonal=-1).bool()
wei = wei.masked_fill(mask, float('-inf'))
wei

tensor([[[[1.5230e-08, 5.3802e-32],
          [      -inf, 1.0000e+00]],

         [[0.0000e+00, 0.0000e+00],
          [      -inf, 1.0000e+00]]],


        [[[0.0000e+00, 0.0000e+00],
          [      -inf, 1.0000e+00]],

         [[0.0000e+00, 0.0000e+00],
          [      -inf, 1.0000e+00]]]])

In [74]:
class MultiHeadAttention (torch.nn.Module):
    def __init__ (self, num_heads, embed_dim):
        super().__init__()
    
    def forward (self, x, kvcache):
        kvcache = (torch.randn(2, 2, 6), torch.randn(2, 2, 6))
        print("kv", kvcache)
        return x
class Block (torch.nn.Module):
    def __init__ (self):
        super().__init__()
        self.sa_heads = MultiHeadAttention(num_heads=4, embed_dim=6)
        self.kvcache = None
    def forward (self, x):
        x = self.sa_heads(x, self.kvcache)
        print("kv", self.kvcache)
        return x
b = Block()
b(torch.randn(2, 2, 6))

kv None


tensor([[[-0.0878, -0.8563, -1.2750, -0.4616,  0.2869,  0.3481],
         [-2.2091,  0.7530, -2.7333,  0.5489,  0.4862,  0.2492]],

        [[ 0.1010, -0.6038, -0.5472,  0.2571, -1.4411, -0.3963],
         [-1.2021, -0.6591,  1.5243,  1.4766, -0.2685, -0.5460]]])